# Stage 1: Xác định số lượng thiếu

### IMPORT PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import DateOffset

### IMPORT DATA

In [ ]:
df_demand = pd.read_excel('DATA_ROUND2_SCHACKFEST2022(1).xlsx',sheet_name='1_DEMAND')
df_demand

,ITEM,JAN_2023,FEB_2023,MAR_2023,APR_2023,MAY_2023,JUN_2023
0,THỨC_ĂN_CHO_HEO,18,22,30,25,24,27
1,THỨC_ĂN_CHO_CÁ,16,21,22,23,23,26


In [ ]:
df_bom = pd.read_excel('DATA_ROUND2_SCHACKFEST2022(1).xlsx',sheet_name='2_BOM')
df_bom

,THÀNH PHẨM,NGUYÊN LIỆU,SỐ LƯỢNG NGUYÊN LIỆU \n(TRÊN 1 TẤN THÀNH PHẨM),ĐƠN VỊ CỦA NGUYÊN LIỆU
0,THỨC_ĂN_CHO_HEO,NGÔ,400,kg
1,THỨC_ĂN_CHO_HEO,KHOAI,200,kg
2,THỨC_ĂN_CHO_HEO,SẮN,200,kg
3,THỨC_ĂN_CHO_HEO,XƯƠNG_ĐỘNG_VẬT,200,kg
4,THỨC_ĂN_CHO_HEO,MEN_VI_SINH,10,lít
5,THỨC_ĂN_CHO_HEO,BAO_BÌ_ĐÓNG_GÓI,40,cái
6,THỨC_ĂN_CHO_CÁ,NGÔ,500,kg
7,THỨC_ĂN_CHO_CÁ,BỘT_ĐỖ_TƯƠNG,300,kg
8,THỨC_ĂN_CHO_CÁ,THÓC,100,kg
9,THỨC_ĂN_CHO_CÁ,XƯƠNG_ĐỘNG_VẬT,100,kg


In [ ]:
df_soh = pd.read_excel('DATA_ROUND2_SCHACKFEST2022(1).xlsx',sheet_name='3_SUPPLY-SOH')
df_soh

,ITEM,TỒN KHO CUỐI NGÀY 31/12/2022,ĐƠN VỊ,HẠN SỬ DỤNG
0,KHOAI,10000,kg,2023-02-12
1,SẮN,3000,kg,2023-02-27
2,XƯƠNG_ĐỘNG_VẬT,2000,kg,2023-01-26
3,BỘT_ĐỖ_TƯƠNG,9000,kg,2023-02-20
4,MEN_VI_SINH,200,lít,2023-01-25
5,NGÔ,12000,kg,2023-02-02
6,THÓC,13000,kg,2023-03-03
7,BAO_BÌ_ĐÓNG_GÓI,1220,cái,NaT
8,THỨC_ĂN_CHO_HEO,7000,kg,2023-01-27
9,THỨC_ĂN_CHO_CÁ,8000,kg,2023-01-30


In [ ]:
df_og = pd.read_excel('DATA_ROUND2_SCHACKFEST2022(1).xlsx',sheet_name='4_SUPPLY-ONGOING')
df_og

,NGUYÊN LIỆU,MÃ ĐƠN ĐẶT HÀNG,NGÀY VỀ,SỐ LƯỢNG,ĐƠN VỊ,HẠN SỬ DỤNG
0,KHOAI,#PO12,2023-01-03,1800,kg,2023-04-13
1,NGÔ,#PO13,2023-01-05,1000,kg,2023-04-15
2,SẮN,#PO14,2023-01-12,1000,kg,2023-04-22


In [ ]:
df_raw = pd.read_excel('DATA_ROUND2_SCHACKFEST2022(1).xlsx',sheet_name='5_ITEM MASTER - RAW')
df_raw = df_raw.rename(columns={'ITEM':'NGUYÊN LIỆU'})
df_raw

,NGUYÊN LIỆU,LOẠI,NHÀ CUNG CẤP,LEADTIME ĐẶT HÀNG (NGÀY),MOQ,Đơn vị,SHELF LIFE ĐƯỢC ƯỚC TÍNH KỂ TỪ NGÀY NHẬP KHO (NGÀY)
0,KHOAI,Nguyên liệu,CÔNG TY TNHH NARGI,7,1000,kg,100.0
1,SẮN,Nguyên liệu,CÔNG TY TNHH NARGI,7,1000,kg,100.0
2,NGÔ,Nguyên liệu,CÔNG TY TNHH NARGI,7,3000,kg,100.0
3,BỘT_ĐỖ_TƯƠNG,Nguyên liệu,CÔNG TY TNHH NARGI,7,1000,kg,100.0
4,THÓC,Nguyên liệu,CÔNG TY TNHH NARGI,7,1000,kg,100.0
5,XƯƠNG_ĐỘNG_VẬT,Nguyên liệu,CÔNG TY CP YFOOD,7,1000,kg,30.0
6,MEN_VI_SINH,Nguyên liệu,CÔNG TY VNIBIO,2,100,lít,30.0
7,BAO_BÌ_ĐÓNG_GÓI,Nguyên liệu,CÔNG TY TNHH WOPACK,7,500,cái,NaN


In [ ]:
df_fg = pd.read_excel('DATA_ROUND2_SCHACKFEST2022(1).xlsx',sheet_name='6_ITEM MASTER - FG')
df_fg

,SKU,LOẠI,LEADTIME SẢN XUẤT \n(NGÀY),SHELF LIFE KỂ TỪ NGÀY SẢN XUẤT\n(NGÀY)
0,THỨC_ĂN_CHO_HEO,Thành phẩm,0,60
1,THỨC_ĂN_CHO_CÁ,Thành phẩm,0,60


### Xử lý bảng

In [ ]:
# Unpivot bảng Demand
df_demand= df_demand.rename(columns={'ITEM':'THÀNH PHẨM'})
df_demand_melt =df_demand.melt('THÀNH PHẨM').rename(columns={'variable':'demand_date','value':'demand_qty'})
df_demand_melt['demand_qty'] = df_demand_melt['demand_qty'].astype('float64') #convert to float64
df_demand_melt = df_demand_melt.replace("JAN_2023", "2023-01-15").replace("FEB_2023", "2023-02-15").replace("MAR_2023","2023-03-15").replace("APR_2023","2023-04-15").replace("MAY_2023","2023-05-15").replace("JUN_2023","2023-06-15")
df_demand_melt['demand_date'] = df_demand_melt['demand_date'].astype('datetime64[ns]') #convert to datadate
df_demand_melt

,THÀNH PHẨM,demand_date,demand_qty
0,THỨC_ĂN_CHO_HEO,2023-01-15,18.0
1,THỨC_ĂN_CHO_CÁ,2023-01-15,16.0
2,THỨC_ĂN_CHO_HEO,2023-02-15,22.0
3,THỨC_ĂN_CHO_CÁ,2023-02-15,21.0
4,THỨC_ĂN_CHO_HEO,2023-03-15,30.0
5,THỨC_ĂN_CHO_CÁ,2023-03-15,22.0
6,THỨC_ĂN_CHO_HEO,2023-04-15,25.0
7,THỨC_ĂN_CHO_CÁ,2023-04-15,23.0
8,THỨC_ĂN_CHO_HEO,2023-05-15,24.0
9,THỨC_ĂN_CHO_CÁ,2023-05-15,23.0


In [ ]:
#Unpivote bảng SOH
df_soh = df_soh.rename(columns={'TỒN KHO CUỐI NGÀY 31/12/2022':'2022-12-31'})
df_soh_melt=pd.melt(df_soh, id_vars=['ITEM','ĐƠN VỊ','HẠN SỬ DỤNG'], value_vars=['2022-12-31']).rename(columns={'variable':'supply_date','value':'supply_quantity'})
df_soh_melt['supply_date'] = df_soh_melt['supply_date'].astype('datetime64[ns]') #convert to datadate
df_soh_melt

,ITEM,ĐƠN VỊ,HẠN SỬ DỤNG,supply_date,supply_quantity
0,KHOAI,kg,2023-02-12,2022-12-31,10000
1,SẮN,kg,2023-02-27,2022-12-31,3000
2,XƯƠNG_ĐỘNG_VẬT,kg,2023-01-26,2022-12-31,2000
3,BỘT_ĐỖ_TƯƠNG,kg,2023-02-20,2022-12-31,9000
4,MEN_VI_SINH,lít,2023-01-25,2022-12-31,200
5,NGÔ,kg,2023-02-02,2022-12-31,12000
6,THÓC,kg,2023-03-03,2022-12-31,13000
7,BAO_BÌ_ĐÓNG_GÓI,cái,NaT,2022-12-31,1220
8,THỨC_ĂN_CHO_HEO,kg,2023-01-27,2022-12-31,7000
9,THỨC_ĂN_CHO_CÁ,kg,2023-01-30,2022-12-31,8000


#### Tách SOH -> SOH-Thành phẩm + SOH-Nguyên liệu

In [ ]:
# soh-thành phẩm
df_sohtp1 = df_soh_melt[df_soh_melt['ITEM'] =='THỨC_ĂN_CHO_HEO']
df_sohtp2 =df_soh_melt[df_soh_melt['ITEM']=='THỨC_ĂN_CHO_CÁ']
df_sohtp = df_sohtp1.merge(df_sohtp2,how=('outer'),on=('ITEM','ĐƠN VỊ','HẠN SỬ DỤNG','supply_date','supply_quantity'))
df_sohtp = df_sohtp.rename(columns={'ITEM':'THÀNH PHẨM'})
df_sohtp['supply_quantity'] = df_sohtp['supply_quantity'] / 1000
df_sohtp['ĐƠN VỊ'] = df_sohtp['ĐƠN VỊ'].replace(['kg'], 'tấn')
df_sohtp

,THÀNH PHẨM,ĐƠN VỊ,HẠN SỬ DỤNG,supply_date,supply_quantity
0,THỨC_ĂN_CHO_HEO,tấn,2023-01-27,2022-12-31,7.0
1,THỨC_ĂN_CHO_CÁ,tấn,2023-01-30,2022-12-31,8.0


In [ ]:
# soh-nguyên liệu
df_sohnl1 = df_soh_melt[df_soh_melt['ITEM'] != 'THỨC_ĂN_CHO_CÁ']
df_sohnl2 = df_soh_melt[df_soh_melt['ITEM'] != 'THỨC_ĂN_CHO_HEO']
df_sohnl = df_sohnl1.merge(df_sohnl2,how=('inner'),on=('ITEM','ĐƠN VỊ','HẠN SỬ DỤNG','supply_date','supply_quantity'))
df_sohnl = df_sohnl.rename(columns={'ITEM':'NGUYÊN LIỆU'})
df_sohnl

,NGUYÊN LIỆU,ĐƠN VỊ,HẠN SỬ DỤNG,supply_date,supply_quantity
0,KHOAI,kg,2023-02-12,2022-12-31,10000
1,SẮN,kg,2023-02-27,2022-12-31,3000
2,XƯƠNG_ĐỘNG_VẬT,kg,2023-01-26,2022-12-31,2000
3,BỘT_ĐỖ_TƯƠNG,kg,2023-02-20,2022-12-31,9000
4,MEN_VI_SINH,lít,2023-01-25,2022-12-31,200
5,NGÔ,kg,2023-02-02,2022-12-31,12000
6,THÓC,kg,2023-03-03,2022-12-31,13000
7,BAO_BÌ_ĐÓNG_GÓI,cái,NaT,2022-12-31,1220


### Demand sau khi trừ tồn kho thành phẩm

In [ ]:
df_ndemand = df_demand_melt.merge(df_sohtp,how='left',on=['THÀNH PHẨM'])
df_ndemand= df_ndemand[df_ndemand['demand_date'] <= df_ndemand['HẠN SỬ DỤNG']]
df_ndemand

,THÀNH PHẨM,demand_date,demand_qty,demand_quantity,ĐƠN VỊ,HẠN SỬ DỤNG,supply_date,supply_quantity
0,THỨC_ĂN_CHO_HEO,2023-01-15,18.0,11.0,tấn,2023-01-27,2022-12-31,7.0
1,THỨC_ĂN_CHO_CÁ,2023-01-15,16.0,8.0,tấn,2023-01-30,2022-12-31,8.0


In [ ]:
df_demand_melt.loc[df_demand_melt['demand_qty'] - df_ndemand['supply_quantity']  < 0, 'demand_quantity'] = 0
df_demand_melt.loc[df_demand_melt['demand_qty'] - df_ndemand['supply_quantity']  >= 0, 'demand_quantity'] = df_demand_melt['demand_qty']-df_ndemand['supply_quantity']
df_demand_melt

,THÀNH PHẨM,demand_date,demand_qty,demand_quantity
0,THỨC_ĂN_CHO_HEO,2023-01-15,18.0,11.0
1,THỨC_ĂN_CHO_CÁ,2023-01-15,16.0,8.0
2,THỨC_ĂN_CHO_HEO,2023-02-15,22.0,NaN
3,THỨC_ĂN_CHO_CÁ,2023-02-15,21.0,NaN
4,THỨC_ĂN_CHO_HEO,2023-03-15,30.0,NaN
5,THỨC_ĂN_CHO_CÁ,2023-03-15,22.0,NaN
6,THỨC_ĂN_CHO_HEO,2023-04-15,25.0,NaN
7,THỨC_ĂN_CHO_CÁ,2023-04-15,23.0,NaN
8,THỨC_ĂN_CHO_HEO,2023-05-15,24.0,NaN
9,THỨC_ĂN_CHO_CÁ,2023-05-15,23.0,NaN


In [ ]:
df_demand_melt['demand_quantity'] = df_demand_melt['demand_quantity'].fillna(df_demand_melt['demand_qty'])
df_newdemand=df_demand_melt[['THÀNH PHẨM','demand_date','demand_quantity']]
df_newdemand

,THÀNH PHẨM,demand_date,demand_quantity
0,THỨC_ĂN_CHO_HEO,2023-01-15,11.0
1,THỨC_ĂN_CHO_CÁ,2023-01-15,8.0
2,THỨC_ĂN_CHO_HEO,2023-02-15,22.0
3,THỨC_ĂN_CHO_CÁ,2023-02-15,21.0
4,THỨC_ĂN_CHO_HEO,2023-03-15,30.0
5,THỨC_ĂN_CHO_CÁ,2023-03-15,22.0
6,THỨC_ĂN_CHO_HEO,2023-04-15,25.0
7,THỨC_ĂN_CHO_CÁ,2023-04-15,23.0
8,THỨC_ĂN_CHO_HEO,2023-05-15,24.0
9,THỨC_ĂN_CHO_CÁ,2023-05-15,23.0


### Merge soh-Nguyên liệu + Ongoing 

In [ ]:
# soh-nguyên liệu + ongoing
df_og = df_og.rename(columns={'NGÀY VỀ':'supply_date','SỐ LƯỢNG':'supply_quantity'})
df_supply = df_sohnl.merge(df_og,how='outer',on=['NGUYÊN LIỆU','ĐƠN VỊ','HẠN SỬ DỤNG','supply_date','supply_quantity'])
df_supply=df_supply[['NGUYÊN LIỆU','ĐƠN VỊ','HẠN SỬ DỤNG','supply_date','supply_quantity']]
df_supply['supply_quantity'] = df_supply['supply_quantity'].astype('float64') #convert to float
df_supply = df_supply.sort_values(by=['NGUYÊN LIỆU','supply_date'])
df_supply['HẠN SỬ DỤNG'] = df_supply['HẠN SỬ DỤNG'].fillna('2050-01-01')
df_supply 

,NGUYÊN LIỆU,ĐƠN VỊ,HẠN SỬ DỤNG,supply_date,supply_quantity
7,BAO_BÌ_ĐÓNG_GÓI,cái,2050-01-01,2022-12-31,1220.0
3,BỘT_ĐỖ_TƯƠNG,kg,2023-02-20,2022-12-31,9000.0
0,KHOAI,kg,2023-02-12,2022-12-31,10000.0
8,KHOAI,kg,2023-04-13,2023-01-03,1800.0
4,MEN_VI_SINH,lít,2023-01-25,2022-12-31,200.0
5,NGÔ,kg,2023-02-02,2022-12-31,12000.0
9,NGÔ,kg,2023-04-15,2023-01-05,1000.0
1,SẮN,kg,2023-02-27,2022-12-31,3000.0
10,SẮN,kg,2023-04-22,2023-01-12,1000.0
6,THÓC,kg,2023-03-03,2022-12-31,13000.0


### MERGE DEMAD + BOM

In [ ]:
df_mdemand = df_newdemand.merge(df_bom,how='left',on='THÀNH PHẨM')
df_mdemand['demandquantity'] = df_mdemand['demand_quantity'] * df_mdemand['SỐ LƯỢNG NGUYÊN LIỆU \n(TRÊN 1 TẤN THÀNH PHẨM)']
df_mdemand = df_mdemand[['NGUYÊN LIỆU','demand_date','demandquantity']]
df_mdemand['cumsum_demand'] = df_mdemand.groupby(['NGUYÊN LIỆU','demand_date'])['demandquantity'].cumsum()
df_mdemand

,NGUYÊN LIỆU,demand_date,demandquantity,cumsum_demand
0,NGÔ,2023-01-15,4400.0,4400.0
1,KHOAI,2023-01-15,2200.0,2200.0
2,SẮN,2023-01-15,2200.0,2200.0
3,XƯƠNG_ĐỘNG_VẬT,2023-01-15,2200.0,2200.0
4,MEN_VI_SINH,2023-01-15,110.0,110.0
...,...,...,...,...
67,BỘT_ĐỖ_TƯƠNG,2023-06-15,7800.0,7800.0
68,THÓC,2023-06-15,2600.0,2600.0
69,XƯƠNG_ĐỘNG_VẬT,2023-06-15,2600.0,8000.0
70,MEN_VI_SINH,2023-06-15,260.0,530.0


# Break bảng

In [ ]:
df_demandMonth1 =df_mdemand[df_mdemand['demand_date'] == '2023-01-15']
df_demandMonth2 =df_mdemand[df_mdemand['demand_date'] == '2023-02-15']
df_demandMonth3 =df_mdemand[df_mdemand['demand_date'] == '2023-03-15']
df_demandMonth4 =df_mdemand[df_mdemand['demand_date'] == '2023-04-15']
df_demandMonth5 =df_mdemand[df_mdemand['demand_date'] == '2023-05-15']
df_demandMonth6 =df_mdemand[df_mdemand['demand_date'] == '2023-06-15']

## Tồn kho

In [ ]:
def tonkho(i):
    i = i.groupby(['NGUYÊN LIỆU']).max()
    df_merged = i.merge(df_supply,how='left',on='NGUYÊN LIỆU') # Demand + Supply
    df_merged = df_merged[df_merged['demand date'] <= df_merged['HẠN SỬ DỤNG']] #Check HSD
    df_merged['cumsum supply'] = df_merged.groupby(['NGUYÊN LIỆU','demand date'])['supply quantity'].cumsum() #cumsum supply
    df_du = df_merged[df_merged['cumsum supply'] >= df_merged['cumsum demand']]
    df_du = df_du[df_du['demand date'] > df_du['supply date']]
    df_stock = df_du.groupby(['NGUYÊN LIỆU']).first()
    df_stock['stock'] = df_stock['cumsum supply'] - df_stock['cumsum demand']
    df_stock = df_stock.merge(i,how='right',on=['NGUYÊN LIỆU','demand date','demand quantity','cumsum demand'])
    df_stock['stock'] = df_stock['stock'].fillna(0)
    df_stock = df_stock.merge(df_du,how='outer',on=['NGUYÊN LIỆU','ĐƠN VỊ','HẠN SỬ DỤNG','supply date','supply quantity','demand date','demand quantity','cumsum demand','cumsum supply'])
    df_stock['stock'] = df_stock['stock'].fillna(df_stock['supply quantity'])
    df_stock = df_stock[['NGUYÊN LIỆU','demand date','supply date','HẠN SỬ DỤNG','stock']]
    return df_stock

In [ ]:
tonkho(df_demandMonth1)

KeyError: 'demand date'

# Thiếu

In [ ]:
def hangthieu(i):
    i = i.groupby(['NGUYÊN LIỆU']).max()
    df_merged = i.merge(df_supply,how='left',on='NGUYÊN LIỆU')
    df_merged = df_merged[df_merged['demand_date'] <= df_merged['HẠN SỬ DỤNG']] #Check HSD
    #df_merged.loc[df_merged['demand_date'] <= df_merged['HẠN SỬ DỤNG'], 'supply_quantity'] = 0
    df_merged['cumsum_supply'] = df_merged.groupby(['NGUYÊN LIỆU','demand_date'])['supply_quantity'].cumsum() #cumsum supply
    df_thieu = df_merged[df_merged['supply_date'] < df_merged['demand_date']]
    df_thieu = df_merged[df_merged['cumsum_supply'] < df_merged['cumsum_demand']]
    df_thieu['SL thiếu'] = df_thieu['cumsum_demand']-df_thieu['cumsum_supply']
    df_thieu = df_thieu[['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu']]
    return df_thieu

In [ ]:
hangthieu(df_mdemand1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9640\1434955758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_thieu['SL thiếu'] = df_thieu['cumsum_demand']-df_thieu['cumsum_supply']


,NGUYÊN LIỆU,demand_date,supply_date,SL thiếu
10,XƯƠNG_ĐỘNG_VẬT,2023-01-15,2022-12-31,1000.0


# Tồn kho tháng khác tháng 1

In [ ]:
def tk(i):
    df_supply = tonkho(df_mdemand[df_mdemand['demand_date'] == str(i['demand_date'] -  DateOffset(months=1))[5:15]])
    df_supply =  df_supply.rename(columns={'stock':'supply_quantity'})
    df_supply = df_supply [['NGUYÊN LIỆU','supply_date','HẠN SỬ DỤNG','supply_quantity']]
    i = i.groupby(['NGUYÊN LIỆU']).max()
    df_merged = i.merge(df_supply,how='right',on=['NGUYÊN LIỆU'])
    df_merged = df_merged[df_merged['demand_date'] <= df_merged['HẠN SỬ DỤNG']] #Check HSD
    #df_merged.loc[df_merged['demand_date'] <= df_merged['HẠN SỬ DỤNG'], 'supply_quantity'] = 0
    #df_merged.loc[df_merged['demand_date'] >= df_merged['HẠN SỬ DỤNG'], 'supply_quantity'] = df_merged['supply_quantity']
    df_merged['cumsum_supply'] = df_merged.groupby(['NGUYÊN LIỆU','demand_date'])['supply_quantity'].cumsum() #cumsum supply
    df_stock = df_merged[df_merged['cumsum_supply'] >= df_merged['cumsum_demand']]
    df_stock = df_stock[df_stock['demand_date'] > df_stock['supply_date']]
    df_stock = df_stock.groupby(['NGUYÊN LIỆU']).first()
    df_stock['stock'] = df_stock['cumsum_supply'] - df_stock['cumsum_demand']
    df_stock = df_stock.merge(i,how='right',on=['NGUYÊN LIỆU','demand_date','demandquantity','cumsum_demand'])
    df_stock['stock'] = df_stock['stock'].fillna(0)
    #df_stock.loc[df_stock['cumsum_demand'] > df_stock['cumsum_supply'], 'stock'] = 0
    df_stock = df_stock[['demand_date','supply_date','HẠN SỬ DỤNG','stock']]
    return df_stock

In [ ]:
tk(df_mdemand2)

,demand_date,supply_date,HẠN SỬ DỤNG,stock
NGUYÊN LIỆU,,,,
BAO_BÌ_ĐÓNG_GÓI,2023-02-15,NaT,NaT,0.0
BỘT_ĐỖ_TƯƠNG,2023-02-15,2022-12-31,2023-02-20,300.0
KHOAI,2023-02-15,NaT,NaT,0.0
MEN_VI_SINH,2023-02-15,NaT,NaT,0.0
NGÔ,2023-02-15,NaT,NaT,0.0
SẮN,2023-02-15,NaT,NaT,0.0
THÓC,2023-02-15,2022-12-31,2023-03-03,10100.0
XƯƠNG_ĐỘNG_VẬT,2023-02-15,NaT,NaT,0.0


In [ ]:
def hangthieu2(i):
    df_supply = tonkho(df_mdemand[df_mdemand['demand_date'] == str(i['demand_date'] -  DateOffset(months=1))[5:15]]) 
    df_supply =  df_supply.rename(columns={'stock':'supply_quantity'})
    df_supply = df_supply [['NGUYÊN LIỆU','supply_date','HẠN SỬ DỤNG','supply_quantity']]
    i = i.groupby(['NGUYÊN LIỆU']).max()
    df_merged = i.merge(df_supply,how='left',on='NGUYÊN LIỆU')
    #df_merged = df_merged[df_merged['demand_date'] <= df_merged['HẠN SỬ DỤNG']] #Check HSD
    df_merged.loc[df_merged['demand_date'] >= df_merged['HẠN SỬ DỤNG'], 'supply_quantity'] = 0
    df_merged['cumsum_supply'] = df_merged.groupby(['NGUYÊN LIỆU','demand_date'])['supply_quantity'].cumsum() #cumsum supply
    #df_thieu = df_thieu.groupby(['NGUYÊN LIỆU']).max()
    df_thieu = df_merged[df_merged['cumsum_supply'] < df_merged['cumsum_demand']]
    df_thieu['không kịp hàng']=np.where(df_thieu['supply_date'] > df_thieu['demand_date'],1,0)
    df_thieu = df_thieu.groupby(['NGUYÊN LIỆU']).max()
    #df_thieu[df_thieu['supply_date'] > df_thieu['demand_date']] 
    df_thieu['SL thiếu'] = df_thieu['cumsum_demand']-df_thieu['cumsum_supply']
    df_thieu = df_thieu[['demand_date','supply_date','SL thiếu']]
    return df_thieu
hangthieu2(df_mdemand3)

,demand_date,supply_date,SL thiếu
NGUYÊN LIỆU,,,
BAO_BÌ_ĐÓNG_GÓI,2023-03-15,NaT,2080.0
BỘT_ĐỖ_TƯƠNG,2023-03-15,2022-12-31,6600.0
KHOAI,2023-03-15,NaT,6000.0
MEN_VI_SINH,2023-03-15,NaT,520.0
NGÔ,2023-03-15,NaT,23000.0
SẮN,2023-03-15,NaT,6000.0
THÓC,2023-03-15,2022-12-31,2200.0
XƯƠNG_ĐỘNG_VẬT,2023-03-15,NaT,8200.0


# Final

### Tồn kho

In [ ]:
df_tonkhofinal = tonkho(df_mdemand1).merge(tk(df_mdemand2),how='outer',on=['NGUYÊN LIỆU','demand_date','HẠN SỬ DỤNG','supply_date','stock']).merge(tk(df_mdemand3),how='outer',on=['NGUYÊN LIỆU','demand_date','HẠN SỬ DỤNG','supply_date','stock']).merge(tk(df_mdemand4),how='outer',on=['NGUYÊN LIỆU','demand_date','HẠN SỬ DỤNG','supply_date','stock']).merge(tk(df_mdemand5),how='outer',on=['NGUYÊN LIỆU','demand_date','HẠN SỬ DỤNG','supply_date','stock']).merge(tk(df_mdemand6),how='outer',on=['NGUYÊN LIỆU','demand_date','HẠN SỬ DỤNG','supply_date','stock'])
df_tonkhofinal.groupby(['demand_date']).max()
df_tonkhofinal

,NGUYÊN LIỆU,demand_date,supply_date,HẠN SỬ DỤNG,stock
0,BAO_BÌ_ĐÓNG_GÓI,2023-01-15,2022-12-31,2050-01-01,460.0
1,BỘT_ĐỖ_TƯƠNG,2023-01-15,2022-12-31,2023-02-20,6600.0
2,KHOAI,2023-01-15,2022-12-31,2023-02-12,7800.0
3,MEN_VI_SINH,2023-01-15,2022-12-31,2023-01-25,10.0
4,NGÔ,2023-01-15,2022-12-31,2023-02-02,3600.0
5,SẮN,2023-01-15,2022-12-31,2023-02-27,800.0
6,THÓC,2023-01-15,2022-12-31,2023-03-03,12200.0
7,XƯƠNG_ĐỘNG_VẬT,2023-01-15,NaT,NaT,0.0
8,KHOAI,2023-01-15,2023-01-03,2023-04-13,1800.0
9,NGÔ,2023-01-15,2023-01-05,2023-04-15,1000.0


### Thiếu

In [ ]:
df_hangthieufinal = hangthieu(df_mdemand1).merge(hangthieu2(df_mdemand2),how='outer',on=['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu']).merge(hangthieu2(df_mdemand3),how='outer',on=['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu']).merge(hangthieu2(df_mdemand3),how='outer',on=['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu']).merge(hangthieu2(df_mdemand4),how='outer',on=['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu']).merge(hangthieu2(df_mdemand5),how='outer',on=['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu']).merge(hangthieu2(df_mdemand6),how='outer',on=['NGUYÊN LIỆU','demand_date','supply_date','SL thiếu'])
df_hangthieufinal.groupby(['demand_date','NGUYÊN LIỆU']).max()

C:\Users\Admin\AppData\Local\Temp\ipykernel_9640\1434955758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_thieu['SL thiếu'] = df_thieu['cumsum_demand']-df_thieu['cumsum_supply']
C:\Users\Admin\AppData\Local\Temp\ipykernel_9640\3524744026.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_thieu['không kịp hàng']=np.where(df_thieu['supply_date'] > df_thieu['demand_date'],1,0)


supply_date  SL thiếu
demand_date NGUYÊN LIỆU                          
2023-01-15  XƯƠNG_ĐỘNG_VẬT   2022-12-31    1000.0
2023-02-15  BAO_BÌ_ĐÓNG_GÓI  2022-12-31    1260.0
            KHOAI            2023-01-03    2600.0
            MEN_VI_SINH      2022-12-31     430.0
            NGÔ              2023-01-05   18300.0
            SẮN              2023-01-12    2600.0
            XƯƠNG_ĐỘNG_VẬT          NaT    6500.0
2023-03-15  BAO_BÌ_ĐÓNG_GÓI         NaT    2080.0
            BỘT_ĐỖ_TƯƠNG     2022-12-31    6600.0
            KHOAI                   NaT    6000.0
            MEN_VI_SINH             NaT     520.0
            NGÔ                     NaT   23000.0
            SẮN                     NaT    6000.0
            THÓC             2022-12-31    2200.0
            XƯƠNG_ĐỘNG_VẬT          NaT    8200.0
2023-04-15  BAO_BÌ_ĐÓNG_GÓI         NaT    1920.0
            BỘT_ĐỖ_TƯƠNG            NaT    6900.0
            KHOAI                   NaT    5000.0
            MEN_VI_SINH             NaT     480.0
            NGÔ                     NaT   21500.0
            SẮN                     NaT    5000.0
            THÓC                    NaT    2300.0
            XƯƠNG_ĐỘNG_VẬT          NaT    7300.0
2023-05-15  BAO_BÌ_ĐÓNG_GÓI         NaT    1880.0
            BỘT_ĐỖ_TƯƠNG            NaT    6900.0
            KHOAI                   NaT    4800.0
            MEN_VI_SINH             NaT     470.0
            NGÔ                     NaT   21100.0
            SẮN                     NaT    4800.0
            THÓC                    NaT    2300.0
            XƯƠNG_ĐỘNG_VẬT          NaT    7100.0
2023-06-15  BAO_BÌ_ĐÓNG_GÓI         NaT    2120.0
            BỘT_ĐỖ_TƯƠNG            NaT    7800.0
            KHOAI                   NaT    5400.0
            MEN_VI_SINH             NaT     530.0
            NGÔ                     NaT   23800.0
            SẮN                     NaT    5400.0
            THÓC                    NaT    2600.0
            XƯƠNG_ĐỘNG_VẬT          NaT    8000.0

# Stage 2: Order

In [ ]:
df_merged2=df_hangthieufinal.merge(df_raw,how='left', on='NGUYÊN LIỆU')
df_merged2.loc[df_merged2['SL thiếu'] <= df_merged2['MOQ'] , 'Số lượng order'] = df_merged2['MOQ']
df_merged2.loc[df_merged2['SL thiếu'] > df_merged2['MOQ'] , 'Số lượng order'] = df_merged2['SL thiếu']
df_merged2['Ngày order']=df_merged2['demand_date'] - pd.to_timedelta(df_merged2['LEADTIME ĐẶT HÀNG (NGÀY)'], unit='d')
df_order=df_merged2[['NGUYÊN LIỆU','NHÀ CUNG CẤP','Số lượng order','Ngày order']]
df_order

,NGUYÊN LIỆU,NHÀ CUNG CẤP,Số lượng order,Ngày order
0,XƯƠNG_ĐỘNG_VẬT,CÔNG TY CP YFOOD,1000.0,2023-01-08
1,BAO_BÌ_ĐÓNG_GÓI,CÔNG TY TNHH WOPACK,1260.0,2023-02-08
2,KHOAI,CÔNG TY TNHH NARGI,2600.0,2023-02-08
3,MEN_VI_SINH,CÔNG TY VNIBIO,430.0,2023-02-13
4,NGÔ,CÔNG TY TNHH NARGI,18300.0,2023-02-08
5,SẮN,CÔNG TY TNHH NARGI,2600.0,2023-02-08
6,XƯƠNG_ĐỘNG_VẬT,CÔNG TY CP YFOOD,6500.0,2023-02-08
7,BAO_BÌ_ĐÓNG_GÓI,CÔNG TY TNHH WOPACK,2080.0,2023-03-08
8,BỘT_ĐỖ_TƯƠNG,CÔNG TY TNHH NARGI,6600.0,2023-03-08
9,KHOAI,CÔNG TY TNHH NARGI,6000.0,2023-03-08


In [ ]:
df_final = pd.pivot_table(df_order, values = 'Số lượng order', index =['NGUYÊN LIỆU','NHÀ CUNG CẤP'], columns = ['Ngày order'])
df_final = df_final.fillna(0)
df_final 

,Ngày order,2023-01-08,2023-02-08,2023-02-13,2023-03-08,2023-03-13,2023-04-08,2023-04-13,2023-05-08,2023-05-13,2023-06-08,2023-06-13
NGUYÊN LIỆU,NHÀ CUNG CẤP,,,,,,,,,,,
BAO_BÌ_ĐÓNG_GÓI,CÔNG TY TNHH WOPACK,0.0,1260.0,0.0,2080.0,0.0,1920.0,0.0,1880.0,0.0,2120.0,0.0
BỘT_ĐỖ_TƯƠNG,CÔNG TY TNHH NARGI,0.0,0.0,0.0,6600.0,0.0,6900.0,0.0,6900.0,0.0,7800.0,0.0
KHOAI,CÔNG TY TNHH NARGI,0.0,2600.0,0.0,6000.0,0.0,5000.0,0.0,4800.0,0.0,5400.0,0.0
MEN_VI_SINH,CÔNG TY VNIBIO,0.0,0.0,430.0,0.0,520.0,0.0,480.0,0.0,470.0,0.0,530.0
NGÔ,CÔNG TY TNHH NARGI,0.0,18300.0,0.0,23000.0,0.0,21500.0,0.0,21100.0,0.0,23800.0,0.0
SẮN,CÔNG TY TNHH NARGI,0.0,2600.0,0.0,6000.0,0.0,5000.0,0.0,4800.0,0.0,5400.0,0.0
THÓC,CÔNG TY TNHH NARGI,0.0,0.0,0.0,2200.0,0.0,2300.0,0.0,2300.0,0.0,2600.0,0.0
XƯƠNG_ĐỘNG_VẬT,CÔNG TY CP YFOOD,1000.0,6500.0,0.0,8200.0,0.0,7300.0,0.0,7100.0,0.0,8000.0,0.0
